In [1]:
import os
os.chdir('/home/ubuntu/ODE_project/paper/linear2D')
print(os.getcwd())
import torch
import torch.nn as nn
import numpy as np
from collections import OrderedDict
from pyDOE import lhs
import time
import math
import ot
from torchdiffeq import odeint
from utils import linear_damped_SHO
from utils import cubic_damped_SHO
from utils import linear_3D
from utils import lorentz,HH
from tool import data_gen_real,cluster,compute_time_error
import torch.optim as optim
import torch.nn.functional as F
from torch import nn
import os
from scipy.integrate import solve_ivp
from scipy.io import loadmat
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph
import matplotlib.pyplot as plt
import logging
logger = logging.getLogger(__name__)
from matplotlib import rcParams
from model import MLP,Linear2D_system
font = {'family' : 'Times New Roman',
'weight' : 'normal',
'size'   : 17,
        }
# Cubic2D
from tool import data_gen_real,cluster
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'LSODA'
integrator_keywords['atol'] = 1e-12
x_init =  np.array([2.0,0.0])
eps = 0.02
full_bs = 50000
T = 10
n_cluster = 8
dim = 2
device = torch.device('cpu')
key = integrator_keywords
model ='Linear2D'
data_x,data_t = data_gen_real(full_bs,T,device,x_init,key,model)

/home/ubuntu/ODE_project/paper/linear2D


/home/ubuntu/.conda/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
ode_model = MLP(nh=500,nin = 1,nout=dim).to(device)
ode_model.load_state_dict(torch.load('results/ckpt_ADO_done.pth'))
time_val = torch.linspace(0,T,50000)
pred_t,mean_error = compute_time_error(ode_model,time_val,data_x,data_t)

In [15]:
ode_model = MLP(nh=500,nin = 1,nout=dim).to(device)
ode_model.load_state_dict(torch.load('results/ckpt_ADO.pth'))
time_val = torch.linspace(0,T,50000)
pred_t,mean_error = compute_time_error(ode_model,time_val,data_x,data_t)

In [2]:
model_param = Linear2D_system(device=device)
model_param.load_state_dict(torch.load('results/ckpt_shoot.pth'))
time_val = torch.linspace(0,T,500)
pred_t,mean_error = compute_time_error(model_param,time_val,data_x,data_t,'ODE')
